In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

sns.set_style('ticks')
sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

## Setup

- Load data
- Add metadata
- Draw gates
- Gate transfected cells

Result from this section: DataFrame `df` representing transfected cells.

Load all lenti collection data collected as of 2024.03.27

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'/'kasey'/'2024.04.05_exp89'/'export'

plates = pd.DataFrame({
    'data_path': [base_path/'293T_control', base_path/'293T_plate1', base_path/'293T_plate2', base_path/'293T_plate3',
                  base_path/'MEF_3_plate1', base_path/'MEF_4-1_plate1', base_path/'MEF_4-1_plate2', base_path/'MEF_4-1_plate3'],
    'yaml_path': [base_path/'kasey_yaml'/'plate_control.yaml', base_path/'kasey_yaml'/'plate01.yaml', base_path/'kasey_yaml'/'plate02.yaml', base_path/'kasey_yaml'/'plate03.yaml',
                  base_path/'kasey_yaml'/'mef_3_plate01.yaml', base_path/'kasey_yaml'/'mef_4-1_plate01.yaml', base_path/'kasey_yaml'/'mef_4-1_plate02.yaml', base_path/'kasey_yaml'/'mef_4-1_plate03.yaml'],
})

output_path = rd.rootdir/'output'/'lenti_BATS'
cache_path = output_path/'data.gzip'

metadata_keys = set()
for p in plates['yaml_path'].unique():
    print(p)
    rd.plot.plot_well_metadata(p)
    metadata_keys.update(rd.flow.load_well_metadata(p).keys())
display(metadata_keys)

In [ ]:
# Load data
data = pd.DataFrame()
if cache_path.is_file(): data = pd.read_parquet(cache_path)
else: 
    channel_list = ['mCherry-A','mRuby2-A','FSC-A','SSC-A','mGL-A']
    data = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data = data[data[c]>0]
    
    data.dropna(inplace=True)
    data.to_parquet(rd.outfile(cache_path))
display(data)

In [ ]:
# Add metadata for constructs
metadata = base.get_metadata(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx')
data = data.merge(metadata, how='left', on='construct')
display(data)

# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
construct_palette = metadata_dict['color']
construct_markers = metadata_dict['markers']

Draw gates based on ungated population, then gate infected cells.

Note that there were no uninfected MEF cells.

In [ ]:
gates = pd.DataFrame()
channel_list = ['mGL-A', 'mCherry-A', 'mRuby2-A']
for channel in channel_list:
    gates[channel] = data[data['construct']=='UT'].groupby(['cell'])[channel].apply(lambda x: x.quantile(0.999))
gates.reset_index(inplace=True)

# Add missing gates
gates.loc[len(gates.index)] = ['MEF',0,0,0,]  
gates.loc[gates['cell']=='MEF', channel_list] = gates.loc[gates['cell']=='293T', channel_list].values
gates.loc[gates['cell']=='MEF', 'mGL-A'] = 2e3

# Indicate which channels are relevant for each experiment
gates.sort_values(['cell'], inplace=True)
gates['marker'] = 'mGL-A'
gates['output'] = 'mRuby2-A'

display(gates)

In [ ]:
# Look at MEF data directly to gate infected cells
plot_df = data[data['cell']=='MEF']
g = sns.displot(data=plot_df, x='mGL-A', y='mRuby2-A', hue='construct', palette=construct_palette,
            row='biorep', col='construct', kind='kde', 
            legend=False, log_scale=True, common_norm=False, fill=False)


Gate data per experiment based on marker expression

In [ ]:
def gate_data(df, gates):
    df = df.copy()
    cell = df['cell'].values[0] # the same for entire df, assuming df = data.groupby('cell')
    gates_dict = gates.set_index('cell').to_dict('dict') # format: column -> {index: value}
    marker = gates_dict['marker'][cell]
    df['expressing'] = df[marker] > gates_dict[marker][cell]
    df['marker'] = df[marker]
    df['output'] = df[gates_dict['output'][cell]]
    return df

data = data.groupby('cell')[data.columns].apply(lambda x: gate_data(x,gates))
data.reset_index(inplace=True, drop=True)
max = 1e6
df = data[(data['expressing']) & (data['output']<max)]
display(df)

In [ ]:
# Bin by marker quantiles
by = ['construct','cell','biorep','dox']
df['bin_marker_quantiles'] = df.groupby(by)['marker'].transform(lambda x: pd.qcut(x, q=20, duplicates='drop'))
quantiles = df.groupby(by+['bin_marker_quantiles'])['marker'].median().rename('bin_marker_quantiles_median').reset_index()
df_quantiles = df.merge(quantiles, how='left', on=by+['bin_marker_quantiles'])
display(df_quantiles)

In [ ]:
df_quantiles['marker'] = df_quantiles['marker'].astype(float)
df_quantiles['output'] = df_quantiles['output'].astype(float)

# Population stats
stat_list = [np.std, sp.stats.gmean, sp.stats.variation]
grouped = df_quantiles.groupby(by=['construct','biorep','cell','dox'])
stats = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats.columns = stats.columns.map(lambda i: base.rename_multilevel_cols(i))
stats['count'] = grouped['output'].count().reset_index()['output']
stats = stats.merge(metadata, how='left', on='construct')
display(stats)

In [ ]:
full_name = {'NT': 'uregulated', 'T': 'circuit'}

In [ ]:
# Line plot with marginal histogram
fig, axes = plt.subplots(1,4, figsize=(10,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.3,0.3,0.3)))
plot_df = df_quantiles[(df_quantiles['miR']=='miRE.FF4') & (df_quantiles['group']=='controller') & (df_quantiles['biorep']==1) & (df_quantiles['design']==1) & (df_quantiles['cell']=='MEF') & (df_quantiles['dox']==1000)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=construct_palette, 
             legend=False, dashes=False, style='construct', markers=construct_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlabel='marker', xlim=(4e3,2e5), ylim=(1e1,2.5e4), )
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['miR']=='miRE.FF4') & (df_quantiles['group']=='controller') & (df_quantiles['biorep']==1) & (df_quantiles['design']==1) & (df_quantiles['cell']=='MEF') & (df_quantiles['dox']==0), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=construct_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax)
ax.set(xlabel='density')

# stat gmean
ax = axes[2]
plot_df = stats[(stats['miR']=='miRE.FF4') & (stats['group']=='controller') & (stats['design']==1) & (stats['cell']=='MEF') & (stats['dox']==1000)]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_gmean', hue='construct', palette=construct_palette, 
                    legend=False, ax=ax, marker=construct_markers[construct], s=12, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric\nmean', xlim=(-0.5, 1.5), xlabel='')
ax.set_xticklabels([full_name[metadata_dict['ts_kind'][c.get_text()]] for c in ax.get_xticklabels()], rotation=90)

# stat std
ax = axes[3]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='output_std', hue='construct', palette=construct_palette, 
                    legend=False, ax=ax, marker=construct_markers[construct], s=12, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard\ndeviation', xlim=(-0.5, 1.5), xlabel='')
ax.set_xticklabels([full_name[metadata_dict['ts_kind'][c.get_text()]] for c in ax.get_xticklabels()], rotation=90)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist-stat_miRE.FF4-design1_MEF.svg')), bbox_inches='tight')

In [ ]:
df_quantiles['bin_marker_quantiles_median_log'] = df_quantiles['bin_marker_quantiles_median'].apply(np.log10)
df_quantiles['output_log'] = df_quantiles['output'].apply(np.log10)

In [ ]:
stat_list = [sp.stats.gmean, ]#sp.stats.gstd, sp.stats.variation]
grouped = df_quantiles.groupby(by=['construct','biorep','cell','dox','bin_marker_quantiles_median'])
stats_quantiles = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats_quantiles.columns = stats_quantiles.columns.map(lambda i: base.rename_multilevel_cols(i))
stats_quantiles['count'] = grouped['output'].count().reset_index()['output']
stats_quantiles = stats_quantiles.merge(metadata, how='left', on='construct')
display(stats_quantiles)

stats_quantiles['bin_marker_quantiles_median_log'] = stats_quantiles['bin_marker_quantiles_median'].apply(np.log10)
stats_quantiles['output_gmean_log'] = stats_quantiles['output_gmean'].apply(np.log10)

In [ ]:
# Compute slope for all constructs
def get_slope(df):
    slope, intercept, r_value, p_value, stderr = sp.stats.linregress(df['bin_marker_quantiles_median_log'], df['output_gmean_log'])
    result = pd.DataFrame(columns=['slope', 'intercept_log', 'r_value', 'p_value', 'stderr'])
    result.loc[len(result.index)] = [slope, intercept, r_value, p_value, stderr]
    return result

fits = stats_quantiles.groupby(['construct','biorep','cell','dox'])[stats_quantiles.columns].apply(get_slope).reset_index()
fits['intercept'] = fits['intercept_log'].apply(lambda x: 10**x)
fits = fits.merge(metadata, how='left', on='construct')
display(fits)

In [ ]:
# MEF miRE-FF4 design 1 slope
fig, axes = plt.subplots(1,3, figsize=(9,5), gridspec_kw=dict(width_ratios=(1,0.3,0.3), wspace=0.35))
plot_df = df_quantiles[(df_quantiles['miR']=='miRE.FF4') & (df_quantiles['group']=='controller') & (df_quantiles['biorep']==1) & (df_quantiles['design']==1) & (df_quantiles['cell']=='MEF') & (df_quantiles['dox']==1000)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=construct_palette, 
             legend=False, dashes=False, style='construct', markers=construct_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=(4e3,2e5), ylim=(1e1,2.5e4), xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['miR']=='miRE.FF4') & (df_quantiles['group']=='controller') & (df_quantiles['biorep']==1) & (df_quantiles['design']==1) & (df_quantiles['cell']=='MEF') & (df_quantiles['dox']==0), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=construct_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax)
ax.set(xlabel='density', ylim=(2e1,1e5), yticklabels=['']*(len(ax.get_yticklabels())-1), ylabel='')

# slope
ax = axes[2]
plot_df = fits[(fits['miR']=='miRE.FF4') & (fits['group']=='controller') & (fits['design']==1) & (fits['cell']=='MEF') & (fits['dox']==1000)]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='construct', y='slope', hue='construct', palette=construct_palette, 
                    legend=False, ax=ax, marker=construct_markers[construct], s=12, jitter=0.2, linewidth=1, edgecolor='white')
    
    x_range = np.linspace(np.log10(4.5e3), np.log10(1.5e5), 1000)
    axes[0].plot(10**(x_range), 10**(group['slope'].values[1] * x_range + group['intercept_log'].values[1]), color='black', zorder=100)

sns.despine(ax=ax)
ax.set(title='Slope', xlim=(-0.5, 1.5), xlabel='', ylabel='', ylim=(0,1.1))
ax.set_xticklabels([full_name[metadata_dict['ts_kind'][c.get_text()]] for c in ax.get_xticklabels()], rotation=90)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist-slope_miRE.FF4-design1-MEF.svg')), bbox_inches='tight')